# Niveau 1 - Détection Heuristique Enrichie

**Objectif principal :** Enrichir l'information de tous les résumés pour alimenter les niveaux suivants (pas de rejet définitif)

## Analyses intégrées (8 types) :
1. Anomalies statistiques (longueur, ponctuation, diversité lexicale)
2. Complexité syntaxique (structure phrases, connecteurs logiques)
3. Répétitions agressives (patterns répétitifs suspects)
4. Densité d'entités (distribution éléments nommés)
5. Incohérences temporelles (anachronismes, contradictions chronologiques)
6. Validation des entités (NER + bases externes)
7. Relations causales suspectes (plausibilité liens cause-effet)
8. Intégration métriques existantes (coherence/factuality scores, grades)



## 1. Configuration et Imports

In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import time
import warnings
import json
warnings.filterwarnings('ignore')

# Configuration des chemins
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"Répertoire projet: {project_root}")
print(f"Répertoire src: {src_path}")

# Import des modules de détection
from detection.level0_prefilter import QualityFilter
from detection.level1_heuristic import Level1HeuristicDetector, quick_heuristic_check

print("\nModules importés avec succès")

Répertoire projet: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector
Répertoire src: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\src

Modules importés avec succès


## 2. Chargement des Données Complètes

In [3]:
# Charger les 372 résumés originaux
data_path = os.path.join(project_root, 'data', 'results', 'batch_summary_production.csv')
df_original = pd.read_csv(data_path)

print(f"Données originales: {len(df_original)} résumés")

# Analyse de la distribution initiale
initial_grades = Counter(df_original['quality_grade'])
print(f"\nDistribution des grades (données complètes):")
for grade, count in sorted(initial_grades.items()):
    print(f"  Grade {grade}: {count} résumés ({count/len(df_original)*100:.1f}%)")

# Charger les résultats du Niveau 0 pour identifier les 2 populations
level0_results_path = os.path.join(project_root, 'data', 'detection', 'level0_filter_results.csv')
if os.path.exists(level0_results_path):
    df_level0 = pd.read_csv(level0_results_path)
    valid_level0_ids = df_level0[df_level0['filter_valid'] == True]['id'].tolist()
    
    print(f"\nAnalyse Niveau 0:")
    print(f"  Résumés validés par Niveau 0: {len(valid_level0_ids)}")
    print(f"  Résumés rejetés par Niveau 0: {len(df_original) - len(valid_level0_ids)}")
else:
    print("\nRésultats Niveau 0 non trouvés, traitement de tous les résumés")
    valid_level0_ids = None

Données originales: 372 résumés

Distribution des grades (données complètes):
  Grade A: 62 résumés (16.7%)
  Grade A+: 60 résumés (16.1%)
  Grade B: 11 résumés (3.0%)
  Grade B+: 158 résumés (42.5%)
  Grade C: 17 résumés (4.6%)
  Grade D: 64 résumés (17.2%)

Analyse Niveau 0:
  Résumés validés par Niveau 0: 319
  Résumés rejetés par Niveau 0: 53


## 3. Préparation des Données pour Analyse 

In [4]:
# Préparer TOUS les résumés (319 validés + 53 rejetés par Niveau 0)
all_summaries_data = []
level0_validated_data = []
level0_rejected_data = []

for idx, row in df_original.iterrows():
    summary_id = f"{row['text_id']}_{row['fusion_strategy']}"
    
    summary_dict = {
        'id': summary_id,
        'text': row['summary'],
        'original_length': row['length'],
        'quality_grade': row['quality_grade'],
        'coherence': row['coherence'],
        'factuality': row['factuality'],
        'composite_score': row['composite_score'],
        'level0_status': 'validated' if (valid_level0_ids is None or summary_id in valid_level0_ids) else 'rejected'
    }
    
    all_summaries_data.append(summary_dict)
    
    # Séparer en 2 populations pour analyse
    if valid_level0_ids is None or summary_id in valid_level0_ids:
        level0_validated_data.append(summary_dict)
    else:
        level0_rejected_data.append(summary_dict)

print(f"\nDonnées préparées pour analyse Niveau 1:")
print(f"  Total: {len(all_summaries_data)} résumés")
print(f"  Validés par Niveau 0: {len(level0_validated_data)} résumés")
print(f"  Rejetés par Niveau 0: {len(level0_rejected_data)} résumés")



Données préparées pour analyse Niveau 1:
  Total: 372 résumés
  Validés par Niveau 0: 319 résumés
  Rejetés par Niveau 0: 53 résumés


In [5]:

# Distribution par statut Niveau 0
print(f"\nDistribution par population:")
for status in ['validated', 'rejected']:
    subset = [s for s in all_summaries_data if s['level0_status'] == status]
    if subset:
        status_grades = Counter([s['quality_grade'] for s in subset])
        print(f"  Population {status}:")
        for grade, count in sorted(status_grades.items()):
            print(f"    Grade {grade}: {count} résumés")


Distribution par population:
  Population validated:
    Grade A: 62 résumés
    Grade A+: 60 résumés
    Grade B: 7 résumés
    Grade B+: 115 résumés
    Grade C: 15 résumés
    Grade D: 60 résumés
  Population rejected:
    Grade B: 4 résumés
    Grade B+: 43 résumés
    Grade C: 2 résumés
    Grade D: 4 résumés


## 4. 

In [6]:
# Prendre un exemple représentatif (Grade A+)
test_summary = level0_validated_data[10] if len(level0_validated_data) > 10 else all_summaries_data[10]
detector_enriched = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")

result_enriched = detector_enriched.detect_hallucinations(
    test_summary['text'], 
    test_summary['id'],
    coherence_score=test_summary['coherence'],
    factuality_score=test_summary['factuality'],
    quality_grade=test_summary['quality_grade']
)

print(f"Résumé testé: {test_summary['id']} (Grade: {test_summary['quality_grade']})")
print(f"   Statut Niveau 0: {test_summary['level0_status']}")
print(f"   Extrait: {test_summary['text'][:150]}...\n")

INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès


Résumé testé: 10_adaptive (Grade: A+)
   Statut Niveau 0: validated
   Extrait: , Un portrait défiguré de l’ex président syrien Bachar al-Assad dans un centre de sécurité gouvernemental saccagé, à Damas, le, le jour du renversemen...



In [7]:
print(f"Évaluation globale:")
print(f"   Statut: {'VALIDE' if result_enriched.is_valid else 'SUSPECT'}")
print(f"   Confiance: {result_enriched.confidence_score:.3f}")
print(f"   Niveau de risque: {result_enriched.risk_level}")
print(f"   Issues détectées: {len(result_enriched.detected_issues)}")
print(f"   Temps de traitement: {result_enriched.processing_time_ms:.1f}ms\n")


Évaluation globale:
   Statut: VALIDE
   Confiance: 1.000
   Niveau de risque: low
   Issues détectées: 1
   Temps de traitement: 53.2ms



In [8]:
# Need to restart the kernel and reimport to get the fixed methods
import importlib
import sys

# Reload the module
if 'detection.level1_heuristic' in sys.modules:
    importlib.reload(sys.modules['detection.level1_heuristic'])

from detection.level1_heuristic import Level1HeuristicDetector

# Re-run the test
test_summary = level0_validated_data[10] if len(level0_validated_data) > 10 else all_summaries_data[10]
detector_enriched = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")

result_enriched = detector_enriched.detect_hallucinations(
    test_summary['text'], 
    test_summary['id'],
    coherence_score=test_summary['coherence'],
    factuality_score=test_summary['factuality'],
    quality_grade=test_summary['quality_grade']
)

print(f"PROFIL STATISTIQUE:")
for key, value in result_enriched.statistical_profile.items():
    print(f"   {key}: {value}")

INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès


PROFIL STATISTIQUE:
   word_count: 27
   sentence_count: 1
   avg_sentence_length: 27.0
   punctuation_ratio: 0.04216867469879518
   lexical_diversity: 0.9259259259259259
   statistical_anomalies_count: 1
   has_length_issues: False
   has_punctuation_issues: True


In [9]:
print(f"\nEXTRACTION D'ENTITÉS:")
entities = result_enriched.entity_extraction
print(f"   Personnes: {len(entities['persons'])} → {[p['text'] for p in entities['persons'][:3]]}")
print(f"   Organisations: {len(entities['organizations'])} → {[o['text'] for o in entities['organizations'][:3]]}")
print(f"   Lieux: {len(entities['locations'])} → {[l['text'] for l in entities['locations'][:3]]}")
print(f"   Total entités: {entities['total_entities']}")
print(f"   Entités suspectes: {entities['suspicious_entities']}")


EXTRACTION D'ENTITÉS:
   Personnes: 0 → []
   Organisations: 0 → []
   Lieux: 1 → ['Damas']
   Total entités: 3
   Entités suspectes: 0


In [10]:
print(f"\nMÉTRIQUES DE COMPLEXITÉ:")
for key, value in result_enriched.complexity_metrics.items():
    print(f"   {key}: {value}")


MÉTRIQUES DE COMPLEXITÉ:
   syntactic_issues_count: 0
   has_length_issues: False
   has_connector_issues: False
   readability_score: 1.0


In [11]:
print(f"\n4. INDICATEURS DE QUALITÉ:")
for key, value in result_enriched.quality_indicators.items():
    print(f"   {key}: {value}")


4. INDICATEURS DE QUALITÉ:
   has_coherence_score: True
   has_factuality_score: True
   has_quality_grade: True
   needs_fact_check: 0.0
   priority_level: low


In [12]:
print(f"\nCANDIDATS FACT-CHECK (Top 5):")
candidates = result_enriched.fact_check_candidates[:5]
for i, candidate in enumerate(candidates, 1):
    print(f"   {i}. Type: {candidate['type']} | Texte: {candidate['text']} | Priorité: {candidate['priority']} | Méthode: {candidate['check_method']}")



CANDIDATS FACT-CHECK (Top 5):


In [13]:
# ===== ANALYSE AVEC CODE CORRIGÉ =====
# Force reload du module corrigé
import importlib
if 'detection.level1_heuristic' in sys.modules:
    importlib.reload(sys.modules['detection.level1_heuristic'])

from detection.level1_heuristic import Level1HeuristicDetector

print("Module rechargé avec les corrections appliquées")
print("Corrections :")
print("- Seuils plus permissifs (0.3 → 0.05)")
print("- Pénalités réduites (70% → 15-30%)")
print("- Enrichissement proactif") 
print("- Validation externe désactivée par défaut")

detector_main = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")

start_time = time.time()
valid_summaries_main, all_results_main = detector_main.process_batch(level0_validated_data)
total_time_main = time.time() - start_time

print(f"Traitement terminé en {total_time_main:.2f}s")
print(f"\nRésultats sur résumés validés Niveau 0 (APRÈS CORRECTIONS):")
print(f"   Total analysé: {len(level0_validated_data)}")
print(f"   Suspects détectés: {len(level0_validated_data) - len(valid_summaries_main)}")
print(f"   Taux de détection: {(len(level0_validated_data) - len(valid_summaries_main))/len(level0_validated_data)*100:.1f}%")
print(f"   Temps moyen: {total_time_main/len(level0_validated_data)*1000:.1f}ms par résumé")

Module rechargé avec les corrections appliquées
Corrections :
- Seuils plus permissifs (0.3 → 0.05)
- Pénalités réduites (70% → 15-30%)
- Enrichissement proactif
- Validation externe désactivée par défaut


INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès
INFO:detection.level1_heuristic:Niveau 1 - Batch traité: 189/319 valides (40.8% rejetés), temps moyen: 70.9ms


Traitement terminé en 52.57s

Résultats sur résumés validés Niveau 0 (APRÈS CORRECTIONS):
   Total analysé: 319
   Suspects détectés: 130
   Taux de détection: 40.8%
   Temps moyen: 164.8ms par résumé


In [14]:
# Vérification objectif performance (APRÈS CORRECTIONS)
processing_times_main = [r.processing_time_ms for r in all_results_main]
target_met = sum(1 for t in processing_times_main if t < 100) / len(processing_times_main) * 100
print(f"   Objectif <100ms: {target_met:.1f}% des résumés")

# Amélioration vs objectif original (49.5%)
improvement = target_met - 49.5
print(f"   Amélioration performance: {improvement:+.1f} points")

# Nouvelles métriques d'enrichissement
total_fact_check_new = sum(len(r.fact_check_candidates) for r in all_results_main)
total_wikidata_new = sum(len(r.validation_hints['wikidata_queries']) for r in all_results_main)
print(f"   Candidats fact-check générés: {total_fact_check_new} (vs 5 avant)")
print(f"   Requêtes Wikidata: {total_wikidata_new} (vs 5 avant)")

   Objectif <100ms: 65.8% des résumés
   Amélioration performance: +16.3 points
   Candidats fact-check générés: 252 (vs 5 avant)
   Requêtes Wikidata: 252 (vs 5 avant)


In [15]:
# Corrélation avec grades de qualité (APRÈS CORRECTIONS)
print(f"\nCorrélation avec grades de qualité (NOUVELLES DONNÉES):")
print(f"{'Grade':<8} {'Suspects':<12} {'%':<8} {'Confiance':<12} {'Temps (ms)'}")
print("-" * 55)

for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_data = [s for s in level0_validated_data if s['quality_grade'] == grade]
    if grade_data:
        grade_results = [all_results_main[i] for i, s in enumerate(level0_validated_data) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if not r.is_valid)
        total_count = len(grade_data)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        avg_time = sum(r.processing_time_ms for r in grade_results) / len(grade_results)
        
        print(f"{grade:<8} {suspect_count}/{total_count:<8} {suspect_count/total_count*100:<8.1f} {avg_confidence:<12.3f} {avg_time:.1f}")

# Comparaison avant/après pour les grades critiques
print(f"\n=== COMPARAISON AVANT/APRÈS CORRECTIONS ===")
print(f"Grade A+ suspects: AVANT 0/60 (0.0%) → APRÈS {sum(1 for i, s in enumerate(level0_validated_data) if s['quality_grade'] == 'A+' and not all_results_main[i].is_valid)}/60")
print(f"Grade D suspects: AVANT 60/60 (100.0%) → APRÈS {sum(1 for i, s in enumerate(level0_validated_data) if s['quality_grade'] == 'D' and not all_results_main[i].is_valid)}/60")

# Sauvegarder les nouveaux résultats
globals()['level0_validated_results'] = all_results_main


Corrélation avec grades de qualité (NOUVELLES DONNÉES):
Grade    Suspects     %        Confiance    Temps (ms)
-------------------------------------------------------
A+       0/60       0.0      1.000        32.9
A        0/62       0.0      1.000        38.2
B+       75/115      65.2     0.368        97.1
B        2/7        28.6     0.331        57.6
C        15/15       100.0    0.000        118.7
D        38/60       63.3     0.095        82.2

=== COMPARAISON AVANT/APRÈS CORRECTIONS ===
Grade A+ suspects: AVANT 0/60 (0.0%) → APRÈS 0/60
Grade D suspects: AVANT 60/60 (100.0%) → APRÈS 38/60


In [16]:
detector_main = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")

start_time = time.time()
valid_summaries_main, all_results_main = detector_main.process_batch(level0_validated_data)
total_time_main = time.time() - start_time

print(f"Traitement terminé en {total_time_main:.2f}s")
print(f"\nRésultats sur résumés validés Niveau 0:")
print(f"   Total analysé: {len(level0_validated_data)}")
print(f"   Suspects détectés: {len(level0_validated_data) - len(valid_summaries_main)}")
print(f"   Taux de détection: {(len(level0_validated_data) - len(valid_summaries_main))/len(level0_validated_data)*100:.1f}%")
print(f"   Temps moyen: {total_time_main/len(level0_validated_data)*1000:.1f}ms par résumé")


INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès
INFO:detection.level1_heuristic:Niveau 1 - Batch traité: 189/319 valides (40.8% rejetés), temps moyen: 71.6ms


Traitement terminé en 53.13s

Résultats sur résumés validés Niveau 0:
   Total analysé: 319
   Suspects détectés: 130
   Taux de détection: 40.8%
   Temps moyen: 166.5ms par résumé


In [17]:
# ===== ANALYSE RÉSUMÉS REJETÉS NIVEAU 0 (AVEC CODE CORRIGÉ) =====

if len(level0_rejected_data) > 0:
    # Utiliser le même détecteur corrigé
    detector_rejected = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")
    
    start_time = time.time()
    valid_rejected, results_rejected = detector_rejected.process_batch(level0_rejected_data)
    processing_time = time.time() - start_time
    
    print(f"Traitement terminé en {processing_time:.2f}s")
    print(f"\nRésultats sur résumés rejetés Niveau 0 (APRÈS CORRECTIONS):")
    print(f"   Total analysé: {len(level0_rejected_data)}")
    print(f"   Suspects détectés: {len(level0_rejected_data) - len(valid_rejected)}")
    print(f"   Taux de détection: {(len(level0_rejected_data) - len(valid_rejected))/len(level0_rejected_data)*100:.1f}%")
    print(f"   Temps moyen: {processing_time/len(level0_rejected_data)*1000:.1f}ms par résumé")
    
    # Distribution des grades pour les rejetés (NOUVELLES DONNÉES)
    print(f"\nDistribution par grade (résumés rejetés Niveau 0) - APRÈS CORRECTIONS:")
    rejected_grades = Counter([s['quality_grade'] for s in level0_rejected_data])
    for grade, count in sorted(rejected_grades.items()):
        suspect_count = sum(1 for i, s in enumerate(level0_rejected_data) 
                           if s['quality_grade'] == grade and not results_rejected[i].is_valid)
        avg_confidence = sum(r.confidence_score for i, r in enumerate(results_rejected) 
                           if level0_rejected_data[i]['quality_grade'] == grade) / max(1, count)
        print(f"   Grade {grade}: {suspect_count}/{count} suspects ({suspect_count/count*100:.1f}%) - Confiance: {avg_confidence:.3f}")
    
    # Sauvegarder pour usage ultérieur
    globals()['level0_rejected_results'] = results_rejected
    print(f"\nRésultats sauvegardés avec les nouvelles données")
    
else:
    print("Aucun résumé rejeté par Niveau 0 à analyser")
    results_rejected = []
    globals()['level0_rejected_results'] = []

INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès
INFO:detection.level1_heuristic:Niveau 1 - Batch traité: 0/53 valides (100.0% rejetés), temps moyen: 180.6ms


Traitement terminé en 22.58s

Résultats sur résumés rejetés Niveau 0 (APRÈS CORRECTIONS):
   Total analysé: 53
   Suspects détectés: 53
   Taux de détection: 100.0%
   Temps moyen: 426.1ms par résumé

Distribution par grade (résumés rejetés Niveau 0) - APRÈS CORRECTIONS:
   Grade B: 4/4 suspects (100.0%) - Confiance: 0.000
   Grade B+: 43/43 suspects (100.0%) - Confiance: 0.088
   Grade C: 2/2 suspects (100.0%) - Confiance: 0.000
   Grade D: 4/4 suspects (100.0%) - Confiance: 0.000

Résultats sauvegardés avec les nouvelles données


In [18]:

# Corrélation avec grades de qualité
print(f"\nCorrélation avec grades de qualité:")
for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_data = [s for s in level0_validated_data if s['quality_grade'] == grade]
    if grade_data:
        grade_results = [all_results_main[i] for i, s in enumerate(level0_validated_data) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if not r.is_valid)
        total_count = len(grade_data)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"   Grade {grade}: {suspect_count}/{total_count} suspects ({suspect_count/total_count*100:.1f}%) - Confiance moy: {avg_confidence:.3f}")

# Sauvegarder pour usage ultérieur
globals()['level0_validated_results'] = all_results_main



Corrélation avec grades de qualité:
   Grade A+: 0/60 suspects (0.0%) - Confiance moy: 1.000
   Grade A: 0/62 suspects (0.0%) - Confiance moy: 1.000
   Grade B+: 75/115 suspects (65.2%) - Confiance moy: 0.368
   Grade B: 2/7 suspects (28.6%) - Confiance moy: 0.331
   Grade C: 15/15 suspects (100.0%) - Confiance moy: 0.000
   Grade D: 38/60 suspects (63.3%) - Confiance moy: 0.095


In [19]:
# ===== VUE D'ENSEMBLE AVEC NOUVELLES DONNÉES =====

# Combiner tous les nouveaux résultats
all_372_summaries = level0_validated_data + level0_rejected_data
all_372_results = level0_validated_results + level0_rejected_results

print(f"RÉSULTATS GLOBAUX (APRÈS CORRECTIONS):")
total_suspects_new = sum(1 for r in all_372_results if not r.is_valid)
detection_rate_new = total_suspects_new/len(all_372_results)*100

print(f"   Total analysé: {len(all_372_summaries)}")
print(f"   Suspects détectés: {total_suspects_new}")
print(f"   Taux de détection global: {detection_rate_new:.1f}%")

print(f"\nCOMPARAISON TAUX DE DÉTECTION:")
print(f"   AVANT corrections: 62.9%")
print(f"   APRÈS corrections: {detection_rate_new:.1f}%")
print(f"   Amélioration: {62.9 - detection_rate_new:+.1f} points")

print(f"\nRÉPARTITION PAR POPULATION (NOUVELLES DONNÉES):")
validated_suspects_new = len(level0_validated_data) - len(valid_summaries_main)
rejected_suspects_new = len(level0_rejected_data) - len(valid_rejected) if len(level0_rejected_data) > 0 else 0

print(f"   Population validée Niveau 0 ({len(level0_validated_data)} résumés):")
print(f"     Suspects détectés: {validated_suspects_new} ({validated_suspects_new/len(level0_validated_data)*100:.1f}%)")

if len(level0_rejected_data) > 0:
    print(f"   Population rejetée Niveau 0 ({len(level0_rejected_data)} résumés):")
    print(f"     Suspects détectés: {rejected_suspects_new} ({rejected_suspects_new/len(level0_rejected_data)*100:.1f}%)")

print(f"\nDISTRIBUTION GLOBALE PAR GRADE (NOUVELLES DONNÉES):")
print(f"{'Grade':<8} {'Total':<8} {'Suspects':<12} {'%':<8} {'Confiance':<12}")
print("-" * 55)

for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_summaries = [s for s in all_372_summaries if s['quality_grade'] == grade]
    if grade_summaries:
        grade_results = [all_372_results[i] for i, s in enumerate(all_372_summaries) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if not r.is_valid)
        total_count = len(grade_summaries)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"{grade:<8} {total_count:<8} {suspect_count:<12} {suspect_count/total_count*100:<8.1f} {avg_confidence:.3f}")

print(f"\nPERFORMANCE GLOBALE (NOUVELLES DONNÉES):")
all_processing_times_new = [r.processing_time_ms for r in all_372_results]
avg_time_new = sum(all_processing_times_new) / len(all_processing_times_new)
max_time_new = max(all_processing_times_new)
target_100ms_global_new = sum(1 for t in all_processing_times_new if t < 100) / len(all_processing_times_new) * 100

print(f"   Temps moyen: {avg_time_new:.1f}ms (vs 116.3ms avant)")
print(f"   Temps maximum: {max_time_new:.1f}ms (vs 388.6ms avant)")
print(f"   Objectif <100ms: {target_100ms_global_new:.1f}% (vs 49.5% avant)")

# Enrichissement global
total_fact_check_global = sum(len(r.fact_check_candidates) for r in all_372_results)
total_wikidata_global = sum(len(r.validation_hints['wikidata_queries']) for r in all_372_results)
print(f"   Candidats fact-check: {total_fact_check_global} (vs 5 avant)")
print(f"   Requêtes Wikidata: {total_wikidata_global} (vs 5 avant)")

print(f"\n{'='*60}")
print(f"SUCCÈS DES CORRECTIONS:")
improvements = []
if detection_rate_new < 50: improvements.append("✓ Taux détection réduit")
if target_100ms_global_new > 60: improvements.append("✓ Performance améliorée") 
if total_fact_check_global > 50: improvements.append("✓ Enrichissement augmenté")
print(f"   {len(improvements)}/3 objectifs atteints")
print(f"   {' | '.join(improvements) if improvements else 'Ajustements nécessaires'}")
print(f"{'='*60}")

RÉSULTATS GLOBAUX (APRÈS CORRECTIONS):
   Total analysé: 372
   Suspects détectés: 183
   Taux de détection global: 49.2%

COMPARAISON TAUX DE DÉTECTION:
   AVANT corrections: 62.9%
   APRÈS corrections: 49.2%
   Amélioration: +13.7 points

RÉPARTITION PAR POPULATION (NOUVELLES DONNÉES):
   Population validée Niveau 0 (319 résumés):
     Suspects détectés: 130 (40.8%)
   Population rejetée Niveau 0 (53 résumés):
     Suspects détectés: 53 (100.0%)

DISTRIBUTION GLOBALE PAR GRADE (NOUVELLES DONNÉES):
Grade    Total    Suspects     %        Confiance   
-------------------------------------------------------
A+       60       0            0.0      1.000
A        62       0            0.0      1.000
B+       158      118          74.7     0.292
B        11       6            54.5     0.211
C        17       17           100.0    0.000
D        64       42           65.6     0.089

PERFORMANCE GLOBALE (NOUVELLES DONNÉES):
   Temps moyen: 87.1ms (vs 116.3ms avant)
   Temps maximum: 322.1ms 

In [20]:


if len(level0_rejected_data) > 0:
    detector_rejected = Level1HeuristicDetector(use_external_validation=False, sensitivity_mode="balanced")
    
    start_time = time.time()
    valid_rejected, results_rejected = detector_rejected.process_batch(level0_rejected_data)
    processing_time = time.time() - start_time
    
    print(f"Traitement terminé en {processing_time:.2f}s")
    print(f"\nRésultats sur résumés rejetés Niveau 0:")
    print(f"   Total analysé: {len(level0_rejected_data)}")
    print(f"   Suspects détectés: {len(level0_rejected_data) - len(valid_rejected)}")
    print(f"   Taux de détection: {(len(level0_rejected_data) - len(valid_rejected))/len(level0_rejected_data)*100:.1f}%")
    print(f"   Temps moyen: {processing_time/len(level0_rejected_data)*1000:.1f}ms par résumé")
    
    # Distribution des grades pour les rejetés
    print(f"\nDistribution par grade (résumés rejetés Niveau 0):")
    rejected_grades = Counter([s['quality_grade'] for s in level0_rejected_data])
    for grade, count in sorted(rejected_grades.items()):
        suspect_count = sum(1 for i, s in enumerate(level0_rejected_data) 
                           if s['quality_grade'] == grade and not results_rejected[i].is_valid)
        print(f"   Grade {grade}: {suspect_count}/{count} suspects ({suspect_count/count*100:.1f}%)")
    
    # Comparaison des raisons de détection
    print(f"\nAnalyse des patterns de détection:")
    rejected_issues = []
    for result in results_rejected:
        rejected_issues.extend(result.detected_issues)
    
    issue_types = Counter([issue.split(':')[0] for issue in rejected_issues])
    print(f"   Types d'issues les plus fréquents (résumés rejetés Niveau 0):")
    for issue_type, count in issue_types.most_common(5):
        print(f"     {issue_type}: {count} cas")
    
    # Sauvegarder pour usage ultérieur
    globals()['level0_rejected_results'] = results_rejected
    print(f"\nRésultats sauvegardés pour analyse ultérieure")
    
else:
    print("Aucun résumé rejeté par Niveau 0 à analyser")
    results_rejected = []
    globals()['level0_rejected_results'] = []

INFO:detection.level1_heuristic:Modèle spaCy fr_core_news_sm chargé avec succès
INFO:detection.level1_heuristic:Niveau 1 - Batch traité: 0/53 valides (100.0% rejetés), temps moyen: 197.0ms


Traitement terminé en 24.78s

Résultats sur résumés rejetés Niveau 0:
   Total analysé: 53
   Suspects détectés: 53
   Taux de détection: 100.0%
   Temps moyen: 467.6ms par résumé

Distribution par grade (résumés rejetés Niveau 0):
   Grade B: 4/4 suspects (100.0%)
   Grade B+: 43/43 suspects (100.0%)
   Grade C: 2/2 suspects (100.0%)
   Grade D: 4/4 suspects (100.0%)

Analyse des patterns de détection:
   Types d'issues les plus fréquents (résumés rejetés Niveau 0):
     Répétition problématique: 318 cas
     Anomalie statistique: 136 cas
     Métrique suspecte: 89 cas
     Complexité suspecte: 69 cas
     Densité entités suspecte: 15 cas

Résultats sauvegardés pour analyse ultérieure


## 7. Vue d'Ensemble - Cartographie Complète des 372 Résumés

In [21]:


# Combiner tous les résultats
all_372_summaries = level0_validated_data + level0_rejected_data
all_372_results = level0_validated_results + level0_rejected_results

print(f"RÉSULTATS GLOBAUX:")
print(f"   Total analysé: {len(all_372_summaries)}")
print(f"   Suspects détectés: {sum(1 for r in all_372_results if not r.is_valid)}")
print(f"   Taux de détection global: {sum(1 for r in all_372_results if not r.is_valid)/len(all_372_results)*100:.1f}%")

print(f"\nRÉPARTITION PAR POPULATION:")
validated_suspects = len(level0_validated_data) - len(valid_summaries_main)
rejected_suspects = len(level0_rejected_data) - len(valid_rejected) if len(level0_rejected_data) > 0 else 0

print(f"   Population validée Niveau 0 ({len(level0_validated_data)} résumés):")
print(f"     Suspects détectés: {validated_suspects} ({validated_suspects/len(level0_validated_data)*100:.1f}%)")

if len(level0_rejected_data) > 0:
    print(f"   Population rejetée Niveau 0 ({len(level0_rejected_data)} résumés):")
    print(f"     Suspects détectés: {rejected_suspects} ({rejected_suspects/len(level0_rejected_data)*100:.1f}%)")

print(f"\nDISTRIBUTION GLOBALE PAR GRADE:")
for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_summaries = [s for s in all_372_summaries if s['quality_grade'] == grade]
    if grade_summaries:
        grade_results = [all_372_results[i] for i, s in enumerate(all_372_summaries) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if not r.is_valid)
        total_count = len(grade_summaries)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"   Grade {grade}: {suspect_count}/{total_count} suspects ({suspect_count/total_count*100:.1f}%) - Confiance: {avg_confidence:.3f}")

print(f"\nPERFORMANCE GLOBALE:")
all_processing_times = [r.processing_time_ms for r in all_372_results]
avg_time = sum(all_processing_times) / len(all_processing_times)
max_time = max(all_processing_times)
target_100ms_global = sum(1 for t in all_processing_times if t < 100) / len(all_processing_times) * 100

print(f"   Temps moyen: {avg_time:.1f}ms")
print(f"   Temps maximum: {max_time:.1f}ms")
print(f"   Objectif <100ms: {target_100ms_global:.1f}% des résumés")





RÉSULTATS GLOBAUX:
   Total analysé: 372
   Suspects détectés: 183
   Taux de détection global: 49.2%

RÉPARTITION PAR POPULATION:
   Population validée Niveau 0 (319 résumés):
     Suspects détectés: 130 (40.8%)
   Population rejetée Niveau 0 (53 résumés):
     Suspects détectés: 53 (100.0%)

DISTRIBUTION GLOBALE PAR GRADE:
   Grade A+: 0/60 suspects (0.0%) - Confiance: 1.000
   Grade A: 0/62 suspects (0.0%) - Confiance: 1.000
   Grade B+: 118/158 suspects (74.7%) - Confiance: 0.292
   Grade B: 6/11 suspects (54.5%) - Confiance: 0.211
   Grade C: 17/17 suspects (100.0%) - Confiance: 0.000
   Grade D: 42/64 suspects (65.6%) - Confiance: 0.089

PERFORMANCE GLOBALE:
   Temps moyen: 89.5ms
   Temps maximum: 513.0ms
   Objectif <100ms: 60.5% des résumés


In [22]:
print(f"   Enrichissement de {len(all_372_summaries)} résumés avec 6 champs utiles")
print(f"   Performance <100ms respectée pour {target_100ms_global:.1f}% des cas")

# Sauvegarder les variables globales pour utilisation ultérieure
globals()['all_372_summaries'] = all_372_summaries
globals()['all_372_results'] = all_372_results

   Enrichissement de 372 résumés avec 6 champs utiles
   Performance <100ms respectée pour 60.5% des cas


## 8. Analyse des Patterns d'Enrichissement

In [23]:
# Analyse des nouvelles informations enrichies
print("ANALYSE DES PATTERNS D'ENRICHISSEMENT\n")

# Échantillon représentatif pour analyse détaillée
sample_size = min(20, len(all_372_results))
sample_indices = np.linspace(0, len(all_372_results)-1, sample_size, dtype=int)

print(f"Analyse sur échantillon de {sample_size} résumés:")

# Analyse des profils statistiques
word_counts = []
complexity_scores = []
entity_densities = []
priority_scores = []

for i in sample_indices:
    result = all_372_results[i]
    summary = all_372_summaries[i]
    
    # Collecter les métriques enrichies
    word_counts.append(result.statistical_profile['word_count'])
    complexity_scores.append(result.complexity_metrics['readability_score'])
    entity_densities.append(result.entity_extraction['total_entities'])
    priority_scores.append(result.get_priority_score())


ANALYSE DES PATTERNS D'ENRICHISSEMENT

Analyse sur échantillon de 20 résumés:


In [24]:

print(f"\nMétriques d'enrichissement:")
print(f"   Longueur moyenne: {np.mean(word_counts):.1f} mots (min: {np.min(word_counts)}, max: {np.max(word_counts)})")
print(f"   Score de complexité moyen: {np.mean(complexity_scores):.3f}")
print(f"   Entités détectées en moyenne: {np.mean(entity_densities):.1f}")
print(f"   Score de priorité moyen: {np.mean(priority_scores):.3f}")



Métriques d'enrichissement:
   Longueur moyenne: 244.2 mots (min: 19, max: 656)
   Score de complexité moyen: 0.869
   Entités détectées en moyenne: 10.9
   Score de priorité moyen: 0.500


In [25]:
# Analyse des candidats fact-check
all_fact_check_candidates = []
for result in all_372_results:
    all_fact_check_candidates.extend(result.fact_check_candidates)

if all_fact_check_candidates:
    candidate_types = Counter([c['type'] for c in all_fact_check_candidates])
    print(f"\nTypes de candidats fact-check identifiés:")
    for ctype, count in candidate_types.most_common(5):
        print(f"   {ctype}: {count} candidats ({count/len(all_fact_check_candidates)*100:.1f}%)")



Types de candidats fact-check identifiés:
   entity: 325 candidats (100.0%)


In [26]:
# ===== SAUVEGARDE RÉSULTATS CORRIGÉS =====

# Créer le DataFrame enrichi complet avec les nouvelles données
enriched_results_df = pd.DataFrame([
    {
        'id': all_372_summaries[i]['id'],
        'level0_status': 'validated' if i < len(level0_validated_data) else 'rejected',
        'original_grade': all_372_summaries[i]['quality_grade'],
        'coherence': all_372_summaries[i]['coherence'],
        'factuality': all_372_summaries[i]['factuality'],
        'heuristic_valid': all_372_results[i].is_valid,
        'confidence_score': all_372_results[i].confidence_score,
        'risk_level': all_372_results[i].risk_level,
        'processing_time_ms': all_372_results[i].processing_time_ms,
        'num_issues': len(all_372_results[i].detected_issues),
        'priority_score': all_372_results[i].get_priority_score(),
        'word_count': all_372_results[i].statistical_profile['word_count'],
        'total_entities': all_372_results[i].entity_extraction['total_entities'],
        'suspicious_entities': all_372_results[i].entity_extraction['suspicious_entities'],
        'fact_check_candidates_count': len(all_372_results[i].fact_check_candidates),
        'needs_fact_check': all_372_results[i].quality_indicators['needs_fact_check'],
        'detected_issues': '; '.join(all_372_results[i].detected_issues[:5])  # Top 5 issues
    }
    for i in range(len(all_372_summaries))
])

# Créer le répertoire de sortie
output_path = os.path.join(project_root, 'data', 'detection')
os.makedirs(output_path, exist_ok=True)

# Sauvegarder les résultats corrigés
results_file = os.path.join(output_path, 'level1_heuristic_corrected_results.csv')
enriched_results_df.to_csv(results_file, index=False)

# Sauvegarder les statistiques corrigées
stats_file = os.path.join(output_path, 'level1_heuristic_corrected_stats.json')

global_stats_corrected = {
    'analysis_summary': {
        'total_analyzed': len(all_372_summaries),
        'level0_validated': len(level0_validated_data),
        'level0_rejected': len(level0_rejected_data),
        'total_suspects': total_suspects_new,
        'global_detection_rate_percent': detection_rate_new,
        'improvement_vs_before': 62.9 - detection_rate_new
    },
    'performance': {
        'avg_time_ms': float(avg_time_new),
        'max_time_ms': float(max_time_new),
        'target_100ms_met_percent': float(target_100ms_global_new),
        'improvement_vs_before': float(target_100ms_global_new - 49.5)
    },
    'enrichment_metrics': {
        'avg_priority_score': float(np.mean([r.get_priority_score() for r in all_372_results])),
        'total_fact_check_candidates': total_fact_check_global,
        'total_wikidata_queries': total_wikidata_global,
        'improvement_fact_check': total_fact_check_global - 5,
        'improvement_wikidata': total_wikidata_global - 5
    },
    'grade_correlation': {
        grade: {
            'total': int(len([s for s in all_372_summaries if s['quality_grade'] == grade])),
            'suspects': int(sum(1 for i, s in enumerate(all_372_summaries) 
                               if s['quality_grade'] == grade and not all_372_results[i].is_valid))
        }
        for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']
        if any(s['quality_grade'] == grade for s in all_372_summaries)
    },
    'corrections_applied': [
        "Seuils plus permissifs (0.3 → 0.05)",
        "Pénalités réduites (70% → 15-30%)", 
        "Détection répétitions moins agressive (5% → 15%)",
        "Enrichissement proactif activé",
        "Validation externe désactivée par défaut",
        "Bonus pour grades élevés (A+, A, B+)"
    ]
}

with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(global_stats_corrected, f, indent=2, ensure_ascii=False)

print(f"=== FICHIERS SAUVEGARDÉS (DONNÉES CORRIGÉES) ===")
print(f"Résultats enrichis: {results_file}")
print(f"Statistiques globales: {stats_file}")
print(f"{len(enriched_results_df)} résumés enrichis avec 17 colonnes de données")
print(f"Taux de détection final: {detection_rate_new:.1f}%")
print(f"Performance moyenne: {avg_time_new:.1f}ms")
print(f"Enrichissement: {total_fact_check_global} candidats fact-check, {total_wikidata_global} requêtes Wikidata")

=== FICHIERS SAUVEGARDÉS (DONNÉES CORRIGÉES) ===
Résultats enrichis: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_corrected_results.csv
Statistiques globales: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_corrected_stats.json
372 résumés enrichis avec 17 colonnes de données
Taux de détection final: 49.2%
Performance moyenne: 87.1ms
Enrichissement: 325 candidats fact-check, 325 requêtes Wikidata


In [27]:

# Top des résumés prioritaires pour Niveau 2
priority_ranking = [(i, r.get_priority_score()) for i, r in enumerate(all_372_results)]
priority_ranking.sort(key=lambda x: x[1], reverse=True)

print(f"\nTOP 5 résumés prioritaires pour Niveau 2:")
for rank, (idx, score) in enumerate(priority_ranking[:5], 1):
    summary = all_372_summaries[idx]
    result = all_372_results[idx]
    print(f"   {rank}. {summary['id']} (Grade: {summary['quality_grade']}) - Priorité: {score:.3f}")
    print(f"      Issues: {len(result.detected_issues)} | Fact-check candidats: {len(result.fact_check_candidates)}")
 


TOP 5 résumés prioritaires pour Niveau 2:
   1. 37_adaptive (Grade: D) - Priorité: 1.000
      Issues: 4 | Fact-check candidats: 0
   2. 11_confidence_weighted (Grade: B+) - Priorité: 1.000
      Issues: 14 | Fact-check candidats: 0
   3. 12_confidence_weighted (Grade: D) - Priorité: 1.000
      Issues: 13 | Fact-check candidats: 1
   4. 13_confidence_weighted (Grade: B+) - Priorité: 1.000
      Issues: 13 | Fact-check candidats: 3
   5. 14_confidence_weighted (Grade: C) - Priorité: 1.000
      Issues: 12 | Fact-check candidats: 0


## 9. Sauvegarde des Résultats Enrichis

In [28]:

# Créer le DataFrame enrichi complet
enriched_results_df = pd.DataFrame([
    {
        'id': all_372_summaries[i]['id'],
        'level0_status': all_372_summaries[i]['level0_status'],
        'original_grade': all_372_summaries[i]['quality_grade'],
        'coherence': all_372_summaries[i]['coherence'],
        'factuality': all_372_summaries[i]['factuality'],
        'heuristic_valid': all_372_results[i].is_valid,
        'confidence_score': all_372_results[i].confidence_score,
        'risk_level': all_372_results[i].risk_level,
        'processing_time_ms': all_372_results[i].processing_time_ms,
        'num_issues': len(all_372_results[i].detected_issues),
        'priority_score': all_372_results[i].get_priority_score(),
        'word_count': all_372_results[i].statistical_profile['word_count'],
        'total_entities': all_372_results[i].entity_extraction['total_entities'],
        'suspicious_entities': all_372_results[i].entity_extraction['suspicious_entities'],
        'fact_check_candidates_count': len(all_372_results[i].fact_check_candidates),
        'needs_fact_check': all_372_results[i].quality_indicators['needs_fact_check'],
        'detected_issues': '; '.join(all_372_results[i].detected_issues[:5])  # Top 5 issues
    }
    for i in range(len(all_372_summaries))
])

# Créer le répertoire de sortie
output_path = os.path.join(project_root, 'data', 'detection')
os.makedirs(output_path, exist_ok=True)

# Sauvegarder les résultats détaillés
results_file = os.path.join(output_path, 'level1_heuristic_enriched_results.csv')
enriched_results_df.to_csv(results_file, index=False)

# Sauvegarder les statistiques globales
stats_file = os.path.join(output_path, 'level1_heuristic_enriched_stats.json')

global_stats = {
    'analysis_summary': {
        'total_analyzed': len(all_372_summaries),
        'level0_validated': len(level0_validated_data),
        'level0_rejected': len(level0_rejected_data),
        'total_suspects': sum(1 for r in all_372_results if not r.is_valid),
        'global_detection_rate_percent': sum(1 for r in all_372_results if not r.is_valid)/len(all_372_results)*100
    },
    'performance': {
        'avg_time_ms': float(np.mean([r.processing_time_ms for r in all_372_results])),
        'max_time_ms': float(np.max([r.processing_time_ms for r in all_372_results])),
        'target_100ms_met_percent': float(sum(1 for r in all_372_results if r.processing_time_ms < 100) / len(all_372_results) * 100)
    },
    'enrichment_metrics': {
        'avg_priority_score': float(np.mean([r.get_priority_score() for r in all_372_results])),
        'total_fact_check_candidates': sum(len(r.fact_check_candidates) for r in all_372_results),
        'total_wikidata_queries': sum(len(r.validation_hints['wikidata_queries']) for r in all_372_results),
        'total_date_checks': sum(len(r.validation_hints['date_checks']) for r in all_372_results)
    },
    'grade_correlation': {
        grade: {
            'total': int(len([s for s in all_372_summaries if s['quality_grade'] == grade])),
            'suspects': int(sum(1 for i, s in enumerate(all_372_summaries) 
                               if s['quality_grade'] == grade and not all_372_results[i].is_valid))
        }
        for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']
        if any(s['quality_grade'] == grade for s in all_372_summaries)
    },
    'level0_comparison': {
        'validated_population': {
            'total': len(level0_validated_data),
            'suspects': validated_suspects,
            'detection_rate_percent': validated_suspects/len(level0_validated_data)*100 if len(level0_validated_data) > 0 else 0
        },
        'rejected_population': {
            'total': len(level0_rejected_data),
            'suspects': rejected_suspects,
            'detection_rate_percent': rejected_suspects/len(level0_rejected_data)*100 if len(level0_rejected_data) > 0 else 0
        }
    }
}

with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(global_stats, f, indent=2, ensure_ascii=False)


In [29]:

print(f"Fichiers sauvegardés:")
print(f"   Résultats enrichis: {results_file}")
print(f"   Statistiques globales: {stats_file}")

print(f"   {len(enriched_results_df)} résumés enrichis avec 17 colonnes de données")
print(f"   Performance moyenne: {global_stats['performance']['avg_time_ms']:.1f}ms")
print(f"   Taux de détection global: {global_stats['analysis_summary']['global_detection_rate_percent']:.1f}%")


Fichiers sauvegardés:
   Résultats enrichis: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_enriched_results.csv
   Statistiques globales: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_enriched_stats.json
   372 résumés enrichis avec 17 colonnes de données
   Performance moyenne: 89.5ms
   Taux de détection global: 49.2%
